In [155]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [156]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-09-27 14:27:06.229 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /Users/wouter/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 13252.95it/s]


In [157]:
len(train), len(valid)

(81, 20)

In [158]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 32, 3]),
 tensor([ 0,  4,  0, 17,  7, 19, 18, 17,  9,  4,  6, 18,  1,  6,  6,  4, 17,  6,
         13, 11,  3, 16,  6, 18,  5,  2,  5,  3, 17, 15,  8, 17]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?


# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [159]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [160]:
# --- Cell #7: eigen LSTM/GRU model (vervang hele cel) ---
import torch
import torch.nn as nn

# 1) Kies je modeltype hier: 'lstm' of 'gru'
model_type = "gru"   # zet op "gru" om GRU te testen

# 2) Hyperparameters die goed werken richting 90%
input_size   = 3
hidden_size  = 256
num_layers   = 2
dropout      = 0.3
bidirectional = True

# 3) Bepaal num_classes robuust
# Probeer uit een batch te lezen; val terug op een vaste waarde als dat (nog) niet kan
try:
    xb, yb = next(iter(trainstreamer))
    num_classes = int(yb.max().item() + 1)
except Exception:
    # Zet een fallback als je hier nog niet bij trainstreamer kunt:
    num_classes = 10  # <-- pas aan als je het weet
    # Tip: voer desnoods later opnieuw uit als trainstreamer al bestaat

class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, num_classes, bidirectional):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        out_dim = hidden_dim * (2 if bidirectional else 1)
        self.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(out_dim, num_classes))

    def forward(self, x):
        # x: (B, T, F) = (batch, tijd, features)
        _, (h, _) = self.lstm(x)
        # h: (layers*dirs, B, H) -> concat laatste laag fwd+bwd
        last = torch.cat([h[-2], h[-1]], dim=-1) if self.lstm.bidirectional else h[-1]
        return self.head(last)

class GRUClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, num_classes, bidirectional):
        super().__init__()
        self.gru = nn.GRU(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        out_dim = hidden_dim * (2 if bidirectional else 1)
        self.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(out_dim, num_classes))

    def forward(self, x):
        _, h = self.gru(x)
        last = torch.cat([h[-2], h[-1]], dim=-1) if self.gru.bidirectional else h[-1]
        return self.head(last)

# 4) Instantieer het gekozen model in variabele 'model' (de rest van je notebook gebruikt 'model')
if model_type.lower() == "gru":
    model = GRUClassifier(input_size, hidden_size, num_layers, dropout, num_classes, bidirectional)
else:
    model = LSTMClassifier(input_size, hidden_size, num_layers, dropout, num_classes, bidirectional)

model

GRUClassifier(
  (gru): GRU(3, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (head): Sequential(
    (0): Dropout(p=0.3, inplace=False)
    (1): Linear(in_features=512, out_features=20, bias=True)
  )
)

Test the model. What is the output shape you need? Remember, we are doing classification!

In [161]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [162]:
accuracy(y, yhat)

0.0

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [163]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [164]:
yhat[0]

tensor([ 0.0558, -0.0646,  0.0697, -0.1329,  0.0578, -0.0298,  0.0402,  0.1818,
         0.0493,  0.1552,  0.1757,  0.0263,  0.0654, -0.0039,  0.1902, -0.0567,
        -0.0891,  0.0789, -0.0220, -0.0067], grad_fn=<SelectBackward0>)

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [165]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(3.0110, grad_fn=<NllLossBackward0>)

In [166]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

Using MPS


Set up the settings for the trainer and the different types of logging you want

In [167]:
settings = TrainerSettings(
    epochs=35, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 8, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 35
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 8, 'delta': 0.0}

In [168]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [169]:
config = ModelConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)


In [170]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "modelname-here")
    mlflow.set_tag("dev", "your-name-here")
    config = ModelConfig(
        input_size=3,
        hidden_size=256,
        num_layers=2,
        output_size=20,
        dropout=0.1,
    )

    model = GRUmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2025-09-27 14:27:35.978 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250927-142735
2025-09-27 14:27:35.978 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 45.65it/s]
2025-09-27 14:27:37.958 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.3735 test 1.9868 metric ['0.3203']
100%|██████████| 81/81 [00:01<00:00, 44.77it/s]
2025-09-27 14:27:39.971 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.4740 test 0.9926 metric ['0.6328']
100%|██████████| 81/81 [00:01<00:00, 46.74it/s]
2025-09-27 14:27:41.885 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.7824 test 0.4939 metric ['0.8406']
100%|██████████| 81/81 [00:01<00:00, 47.18it/s]
2025-09-27 14:27:43.798 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.3142 test 0.2951 metric ['0.9219']
100%|██████████| 81/81 [00:01<00:00, 46.23it/s]
2025-09-27 14:27:45.74

Try to update the code above by changing the hyperparameters.
    
To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [171]:
trainer.loop() # if you want to pick up training, loop will continue from the last epoch

100%|██████████| 81/81 [00:01<00:00, 46.74it/s]
2025-09-27 14:28:21.412 | INFO     | mltrainer.trainer:report:175 - Resuming epochs from previous training at 45
2025-09-27 14:28:21.431 | INFO     | mltrainer.trainer:report:209 - Epoch 45 train 0.0029 test 0.0148 metric ['0.9953']
2025-09-27 14:28:21.431 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.0085, current loss 0.0148.Counter 9/8.
2025-09-27 14:28:21.432 | INFO     | mltrainer.trainer:loop:103 - Interrupting loop due to early stopping patience.
2025-09-27 14:28:21.432 | INFO     | mltrainer.trainer:loop:108 - early_stopping_save was false, using latest model.Set to true to retrieve best model.
  0%|          | 0/35 [00:01<?, ?it/s]


In [172]:
mlflow.end_run()